# DGA Detection - Data Processing

In [1]:
%pip install --upgrade pip
%pip install pandas scikit-learn scipy numpy alibi tldextract pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import tldextract
import re

from scipy.stats import entropy
from sklearn.model_selection import train_test_split 
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

## CIC Bell DNS 2021 data set

Write out all domain names.

In [11]:
files = ["CICBellDNS2021_CSV_benign.csv", "CICBellDNS2021_CSV_malware.csv", "CICBellDNS2021_CSV_phishing.csv", "CICBellDNS2021_CSV_spam.csv"]

for file in files:
    domains = []
    with open(f"data/cic/{file}") as f:
        for line in f:
            txt = line.replace(' ', '')
            x = re.split(",(?![^\[\]]*(?:\])|[^()]*\))", txt)
            if "Domain" != x[4]:
                domains.append(x[4])
    print(domains[:5])
    # domains = list(dict.fromkeys(domains)) ->  What?
    

["b'google.com.'", "b'google.com.'", "b'www.google.com.'", "b'www.google.com.'", "b'facebook.com.'"]
["b'dicrophani.com.'", "b'dionneg.com.'", "b'vipprojects.cn.'", "b'hhj3.cn.'", "b'hhj3.cn.'"]
["b'programafidelidadeitacard2.cf.'", "b'programafidelidadeitacard2.cf.'", "b'professorjosesilveira.com.'", "b'www.ksylitol.com.'", "b'www.ksylitol.com.'"]
["b'0900259.com.'", "b'koitera.net.'", "b'koitera.com.'", "b'pc.koitera.com.'", "b'0901360.com.'"]
